In [ ]:
%pip install requests beautifulsoup4
import requests
import pprint
import json
from datetime import date, datetime

from bs4 import BeautifulSoup

In [ ]:
URL = "https://news.ycombinator.com/"
resp = requests.get(URL,
                    # params={"day": datetime.today().date(), "p": 1})
                    params={"day":"2023-05-02"})
print(resp.json)
soup = BeautifulSoup(resp.text, 'html.parser')
rows = soup.find_all('tr', class_='athing')


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
cols = "id date rank title url points comments".split()
df = pd.DataFrame(columns=cols,)

def get_posts_for_date(datestring):
    URL = "https://news.ycombinator.com/"
    resp = requests.get(URL,params={"day":datestring})
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = soup.find_all('tr', class_='athing')
    for row in rows:
        id = int(row['id'])
        rank = int(row.find('span',class_="rank").text[:-1])
        title_element = row.find("span", class_='titleline')
        title = title_element.text if title_element else ''
        url = title_element.a["href"] if "https" in title_element.a["href"] else ''
        
        next_row = row.find_next_sibling('tr')
        subline = next_row.find("span", class_="subline")
        if not subline:
            continue
        else:
            subline = subline.text.split()
        if "points" in subline:
            points = int(subline[subline.index("points")-1]) 
        else:
            points = -999
        comments = ""
        if "comments" in subline:
            comments = int(subline[subline.index("comments")-1])
        else:
            points = -999

        value = [id, datestring, rank, title, url, points, comments]
        df.loc[len(df)] = value
    # return df
    

from datetime import date
from time import sleep

for d in range(1,31):
    datestring = date(2023,5,d).isoformat()
    get_posts_for_date(datestring)
    # date_df.to_csv(f"posts-{datestring}.csv")
# df.tail()

In [ ]:
df[["viewed","liked"]] = 0
df.to_csv("data.csv")
df